.csv uzantılı dosyanın yüklemesinin gerçekleştirilmesi

In [26]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='yemek_chatbot_veriseti.csv')
data = loader.load()

In [27]:
len(data)

1350

Veriyi parçalara ayırma (Chunking)

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total",len(docs))

Total 1350


In [29]:
docs [7]

Document(metadata={'source': 'yemek_chatbot_veriseti.csv', 'row': 7}, page_content='text: Günaydın\nintent: selamlama')

Google Generative AI Embeddings'i kullanarak Embedding oluşturma kısmı
- Text veriyi sayısal veriye çevirme

In [30]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello world!")
vector[:5]

[0.04625185579061508,
 -0.04018166661262512,
 -0.026706203818321228,
 -0.03015909716486931,
 0.014996296726167202]

In [31]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [32]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 7}
)

In [33]:
retieved_docs = retriever.invoke("diyet önerisi?")

In [34]:
len(retieved_docs)

7

In [35]:
print(retieved_docs[5].page_content)

text: Sağlıklı diyet menüsü verir misin?
intent: diyet_onerisi


In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=500
)

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [63]:
system_prompt = (
    "You are a helpful assistant specialized in meal recommendations."
    "Use the provided context to answer user questions about recipes and meals."
    "Keep your answers friendly and give detailed sentences."
    "\n\n"
    "{context}"
)

In [64]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [65]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [66]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [67]:
response = rag_chain.invoke({"input":"tatlı önerisi?"})

In [68]:
print(response)

{'input': 'tatlı önerisi?', 'context': [Document(id='775bd154-1289-4438-95b1-f6d530796cb1', metadata={'row': 642, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri olur mu?\nintent: tatli'), Document(id='9632275b-e5cf-4322-9092-15f3ff4abd3e', metadata={'row': 642, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Sağlıklı tatlı önerileri olur mu?\nintent: tatli'), Document(id='d6f1e02e-9ada-4857-85b1-a11463120d40', metadata={'row': 680, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Çikolatalı tatlı nasıl yapılır? lütfen\nintent: tatli'), Document(id='f8931764-3154-44c0-8d46-83aecf9f7d0b', metadata={'row': 639, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Çikolatalı tatlı nasıl yapılır? lütfen\nintent: tatli'), Document(id='3a4c006e-bfd6-4d6d-b439-871e6f38190c', metadata={'row': 680, 'source': 'yemek_chatbot_veriseti.csv'}, page_content='text: Çikolatalı tatlı nasıl yapılır? lütfen\nintent: tatli'), Document(i

In [69]:
print(response["answer"])

Elbette, size sağlıklı tatlılar önerebilirim ve çikolatalı tatlıların nasıl yapıldığı hakkında bilgi verebilirim.
